<a href="https://colab.research.google.com/github/Gireesheruvaram/LLM-Beginer/blob/main/Kata1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install faiss-cpu transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 61.4 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load pre-trained model and tokenizer
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

def generate_embedding(text: str) -> torch.Tensor:
    tokens = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**tokens)
    # Use mean pooling for embeddings
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.squeeze()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [ ]:
# Initialize and Insert Embeddings into FAISS
import faiss
import numpy as np

# Initialize FAISS index
embedding_dimension = 384  # Dimension for `all-MiniLM-L6-v2` model
index = faiss.IndexFlatL2(embedding_dimension)

# Sample documents
documents = [
    "The quick brown fox jumps over the lazy dog.",
    "A journey of a thousand miles begins with a single step.",
    "To be or not to be, that is the question.",
    "All that glitters is not gold.",
]

# Generate embeddings for documents
doc_embeddings = []
for doc in documents:
    emb = generate_embedding(doc).numpy()
    doc_embeddings.append(emb)

doc_embeddings = np.vstack(doc_embeddings)

# Add embeddings to FAISS index
index.add(doc_embeddings)


In [ ]:
def cosine_similarity_search(query: str, top_n: int = 3):
    # Convert query to embedding
    query_embedding = generate_embedding(query).numpy().reshape(1, -1)

    # Perform FAISS search
    distances, indices = index.search(query_embedding, top_n)

    # Fetch corresponding documents
    results = [{"document": documents[idx], "distance": dist} for dist, idx in zip(distances[0], indices[0])]
    return results


In [ ]:
query = "gold and glitter"
results = cosine_similarity_search(query, top_n=2)

print("Query:", query)
print("\nTop Matches:")
for result in results:
    print(f"Document: {result['document']}\nDistance: {result['distance']:.4f}\n")


Query: gold and glitter

Top Matches:
Document: All that glitters is not gold.
Distance: 17.5537

Document: To be or not to be, that is the question.
Distance: 55.8270

